In [1]:
import keras
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import misc
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
import pickle
%matplotlib inline

Using TensorFlow backend.


In [2]:
test_img_fnames = os.listdir('data/test')
train_img_fnames = os.listdir('data/train')
percentage_data_size = 5 / 100
print(len(test_img_fnames), len(train_img_fnames))

2500 22500


# Load in the data
First we need to write a function to load into memory each of the images, and then resize them to (224, 224, 3). scipy.misc.imread and scipy.misc.imresize will be useful for this. You should also rescale the images so that they are on a scale of 0 to 1, meaning each pixel value should be between 0 and 1. This can be achieved by dividing by an appropriate constant.

In [3]:
def load_img(filename):
    img = misc.imread(filename)
    img = misc.imresize(img, (224,224))
    return img / 255

In [4]:
test_imgs = [load_img(os.path.join('data/test', fname)) for fname in test_img_fnames[:int(len(test_img_fnames) * percentage_data_size)]]
test_imgs = np.stack(test_imgs)

In [5]:
train_imgs = [load_img(os.path.join('data/train', fname)) for fname in train_img_fnames[:int(len(train_img_fnames) * percentage_data_size)]]
train_imgs = np.stack(train_imgs)

In [6]:
with open('data/train_labels.pkl', 'rb') as f:
    train_labels = pickle.load(f)

# Load Pretrained VGG network
Now in order to generate the CNN featurizations of our images we need to load a pretrained network. Note that running this network will take a long time on CPU so you can feel free to skip this section and just load the featurizations I provided in the next section

In [ ]:
VGG16 = keras.applications.vgg16.VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

In [ ]:
VGG16.summary()

In [ ]:
featurized_training_data = VGG16.predict(train_imgs, verbose=1)

In [ ]:
featurized_test_data = VGG16.predict(test_imgs, verbose=1)

In [45]:
# Save featurizations, I changed filenames so as not to overwrite the original
import pickle
with open('featurized_train_imgs_gen.pkl', 'wb') as f:
    pickle.dump(featurized_training_data, f)
with open('featurized_test_imgs_gen.pkl', 'wb') as f:
    pickle.dump(featurized_test_data, f)

KeyboardInterrupt: 

# Load featurizations

In [7]:
import pickle
with open('featurized_train_imgs.pkl', 'rb') as f:
    featurized_training_data = pickle.load(f)
with open('featurized_test_imgs.pkl', 'rb') as f:
    featurized_test_data = pickle.load(f)

# Create you network
Now we need to create a network to take in the featurizations and output a label of dog or not dog. To do this you should use Keras' Sequential model. We will need to flatten our (7,7,512) feature input into a vector (HINT: lookup flatten in keras documentation) and then add a Dense layer with some number of neurons (play around with the number of neurons to improve your performance). Then finally we need a Dense layer with 1 neuron and a sigmoid activation to represent our label output. You might want to use more or less model.add calls than have been provided

In [41]:
model = Sequential()
model.add(keras.layers.Flatten(input_shape=(7,7,512)))
model.add(keras.layers.Dense(64, activation="tanh"))
model.add(keras.layers.Dense(32, activation="sigmoid"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 64)                1605696   
_________________________________________________________________
dense_12 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 33        
Total params: 1,607,809
Trainable params: 1,607,809
Non-trainable params: 0
_________________________________________________________________


# Now we need to train the network
You need to compile the model first, and then use the fit function. You should use binary crossentropy as your loss.

In [43]:
model.compile(optimizer="sgd", loss="binary_crossentropy")

In [44]:
model.fit(featurized_training_data, train_labels, epochs=20)

Epoch 1/20
10000/10000 [==============================] - 8s - loss: 0.5105     
Epoch 2/20
10000/10000 [==============================] - 9s - loss: 0.3497     
Epoch 3/20
10000/10000 [==============================] - 8s - loss: 0.2977     
Epoch 4/20
10000/10000 [==============================] - 8s - loss: 0.2705     
Epoch 5/20
10000/10000 [==============================] - 8s - loss: 0.2492     
Epoch 6/20
10000/10000 [==============================] - 8s - loss: 0.2297     
Epoch 7/20
10000/10000 [==============================] - 8s - loss: 0.2163     
Epoch 8/20
10000/10000 [==============================] - 8s - loss: 0.2114     
Epoch 9/20
10000/10000 [==============================] - 8s - loss: 0.1963     
Epoch 10/20
10000/10000 [==============================] - 8s - loss: 0.1878     
Epoch 11/20
10000/10000 [==============================] - 8s - loss: 0.1782     
Epoch 12/20
10000/10000 [==============================] - 8s - loss: 0.1759     
Epoch 13/20
10000/10000 [

Now we need to predict labels for the test set and output it to file. Use keras predict for this. Remember that the predictions are real values between 0 and 1 and you should be outputting just 0 or 1, not a value between.

In [46]:
predictions = model.predict(featurized_test_data)

In [51]:
i = 2
with open('test_labels_%s.txt' % i, 'w') as f:
    f.write("Id,Label\n")
    for i, val in enumerate(predictions):
        prediction = 0 if val < .5 else 1
        f.write(str(i))
        f.write(',')
        f.write(str(prediction))
        f.write('\n')

In [54]:
# Add disagreeing images manually here. 
known_dic = {0:1, 16:1}
def compare_files(f1, f2):
    f1 = open(f1,'r')
    f2 = open(f2,'r')
    f1.readline()
    f2.readline()
    count = 0
    netdif = 0
    unknowns = 0
    for line_num in range(2500):
        mod_1 = f1.readline()
        pred_1 = int(mod_1[mod_1.index(",")+1:][:-1])
        
        mod_2 = f2.readline()
        pred_2 = int(mod_2[mod_2.index(",")+1:][:-1])
        if pred_2 != pred_1:
            netdif += 1
            if line_num in known_dic:
                if known_dic[line_num] == pred_1:
                    count += 1
                else:
                    count -= 1
            else:
                print("Disagreement on %s , with unknown result" % line_num)
                unknowns += 1
    print("model 1 scored " + str(count) + " more than model 2")
    print("Net number of different files is " + str(netdif))
    print("Number of unknown lines is " + str(unknowns))
compare_files('test_labels_2.txt','test_labels_1.txt')
        

Disagreement on 23 , with unknown result
Disagreement on 59 , with unknown result
Disagreement on 79 , with unknown result
Disagreement on 85 , with unknown result
Disagreement on 89 , with unknown result
Disagreement on 93 , with unknown result
Disagreement on 107 , with unknown result
Disagreement on 116 , with unknown result
Disagreement on 132 , with unknown result
Disagreement on 154 , with unknown result
Disagreement on 174 , with unknown result
Disagreement on 195 , with unknown result
Disagreement on 202 , with unknown result
Disagreement on 215 , with unknown result
Disagreement on 271 , with unknown result
Disagreement on 279 , with unknown result
Disagreement on 296 , with unknown result
Disagreement on 322 , with unknown result
Disagreement on 335 , with unknown result
Disagreement on 350 , with unknown result
Disagreement on 351 , with unknown result
Disagreement on 366 , with unknown result
Disagreement on 378 , with unknown result
Disagreement on 397 , with unknown resul

# Part 2

In [23]:
from sklearn.decomposition import PCA

Now we want to run PCA on our images and our featurizations to see the difference. To save time we will just run it on the first 500 images. Take the first 500 images and 500 featurizations and reshape them into shape (500, x).

In [ ]:
original_imgs = ???
featurized = ???

Run PCA with 2 components on the original images

In [ ]:
pca_orig = ???
# YOUR CODE HERE

Run PCA with 2 components on the featurizations

In [ ]:
pca_featurized = ???
# YOUR CODE HERE

Project the original images and featurizations onto the 2 principal components. (HINT: look at the PCA.transform function)

In [ ]:
projected_orig_imgs = ???
projected_featurized = ???

Find the indices of the labels that are cats and the indices that are dogs. np.where will make this very easy

In [ ]:
labels = train_labels[:500]
cat_inds = ???
dog_inds = ???

Plot the PCA of the original images and the PCA of the featurization

In [ ]:
plt.scatter(projected_orig_imgs[cat_inds ,0], projected_orig_imgs[cat_inds, 1], c='red')
plt.scatter(projected_orig_imgs[dog_inds, 0], projected_orig_imgs[dog_inds, 1], c='green')

In [ ]:
plt.scatter(projected_featurized[cat_inds ,0], projected_featurized[cat_inds, 1], c='red')
plt.scatter(projected_featurized[dog_inds, 0], projected_featurized[dog_inds, 1], c='green')